In [1]:
!pip install mysql-connector-python

In [2]:
#Connection to DB
import mysql.connector
from mysql.connector import Error
from datetime import datetime, date, timedelta
import csv
import itertools

def get_connection(db=True):
    """
    Returns a mysql.connector connection.
    If db=False, connects to server only (used to create DB).
    """
    try:
        if db:
            conn = mysql.connector.connect(
                host="localhost",
                user="root",
                password="12345678",
                database="mcd_scheduler"
            )
        else:
            conn = mysql.connector.connect(
                host="localhost",
                user="root",
                password="12345678"
            )
        return conn
    except Error as e:
        print("Error connecting:", e)
        return None

In [3]:
# create database and tables
def create_database():
    conn = None
    try:
        conn = get_connection(db=False)
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS mcd_scheduler")
        print("Database created mcd_scheduler")
    except Error as e:
        print("Error creating database:", e)
    finally:
        if conn and conn.is_connected():
            cursor.close()
            conn.close()

def create_tables():
    conn = None
    try:
        conn = get_connection()
        cursor = conn.cursor()

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS employees(
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(100) NOT NULL,
            role VARCHAR(50),
            phone VARCHAR(50),
            email VARCHAR(100)
        )
        """)

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS shifts(
            id INT AUTO_INCREMENT PRIMARY KEY,
            start_datetime DATETIME NOT NULL,
            end_datetime DATETIME NOT NULL,
            role_required VARCHAR(50)
        )
        """)

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS assignments(
            id INT AUTO_INCREMENT PRIMARY KEY,
            shift_id INT NOT NULL,
            employee_id INT NOT NULL,
            FOREIGN KEY (shift_id) REFERENCES shifts(id) ON DELETE CASCADE,
            FOREIGN KEY (employee_id) REFERENCES employees(id) ON DELETE CASCADE,
            UNIQUE KEY unique_assignment (shift_id, employee_id)
        )
        """)

        conn.commit()
        print("Tables created (employees, shifts, assignments)")
    except Error as e:
        print("Error creating tables:", e)
    finally:
        if conn and conn.is_connected():
            cursor.close()
            conn.close()

# Run these once
create_database()
create_tables()

✅ Database created (or already exists): mcd_scheduler
✅ Tables created (employees, shifts, assignments)


In [4]:
# insert employees
def add_employee(name, role, phone, email):
    conn = None
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO employees (name, role, phone, email) VALUES (%s, %s, %s, %s)",
            (name, role, phone, email)
        )
        conn.commit()
        print("Added:", name)
    except Error as e:
        # ignore duplicate or other errors for repeated runs
        print("Error adding employee:", e)
    finally:
        if conn and conn.is_connected():
            cursor.close()
            conn.close()

employees = [
    ("Frank","Crew","07123456701","frank@example.com"),
    ("Sam","Crew","07123456702","sam@example.com"),
    ("Alice","Crew","07123456703","alice@example.com"),
    ("Bob","Crew","07123456704","bob@example.com"),
    ("Charlie","Crew","07123456705","charlie@example.com"),
    ("David","Crew","07123456706","david@example.com"),
    ("Eva","Crew","07123456707","eva@example.com"),
    ("Rachel","Manager","07123456708","rachel@example.com"),
    ("Grace","Manager","07123456709","grace@example.com"),
    ("Hannah","Manager","07123456710","hannah@example.com"),
    ("Isha","Crew","07123456711","isha@example.com"),
    ("Jai","Crew","07123456712","jai@example.com"),
]

for emp in employees:
    add_employee(*emp)

Added: Frank
Added: Sam
Added: Alice
Added: Bob
Added: Charlie
Added: David
Added: Eva
Added: Rachel
Added: Grace
Added: Hannah
Added: Isha
Added: Jai


In [5]:
# generate shifts
def create_shift(start_dt, end_dt, role_required):
    conn = None
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO shifts (start_datetime, end_datetime, role_required) VALUES (%s, %s, %s)",
            (start_dt, end_dt, role_required)
        )
        conn.commit()
        # no print for each to keep output compact
    except Error as e:
        print("Error creating shift:", e)
    finally:
        if conn and conn.is_connected():
            cursor.close()
            conn.close()

# date range inclusive
start_date = date(2025,9,21)
end_date = date(2025,10,11)

d = start_date
while d <= end_date:
    # morning crew: 08:00 - 16:00
    start_dt = datetime.combine(d, datetime.min.time()).replace(hour=8)
    end_dt = start_dt + timedelta(hours=8)
    create_shift(start_dt, end_dt, "Crew")

    # evening crew: 16:00 - 00:00 (which is next day 00:00)
    start_dt2 = datetime.combine(d, datetime.min.time()).replace(hour=16)
    end_dt2 = start_dt2 + timedelta(hours=8)  # this goes to next day midnight
    create_shift(start_dt2, end_dt2, "Crew")

    # manager shift on weekdays (Mon-Fri)
    if d.weekday() < 5:  # 0=Monday ... 4=Friday
        m_start = datetime.combine(d, datetime.min.time()).replace(hour=8)
        m_end = m_start + timedelta(hours=8)
        create_shift(m_start, m_end, "Manager")

    d = d + timedelta(days=1)

print("✅ Shifts created for range:", start_date, "to", end_date)

✅ Shifts created for range: 2025-09-21 to 2025-10-11


In [6]:
# assign employees to shifts with rules
def get_employees_by_role(role):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT id FROM employees WHERE role=%s ORDER BY id", (role,))
    rows = [r[0] for r in cursor.fetchall()]
    cursor.close()
    conn.close()
    return rows

def employee_has_overlap(employee_id, new_start, new_end):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT COUNT(*) FROM assignments a
        JOIN shifts s ON a.shift_id = s.id
        WHERE a.employee_id=%s
          AND NOT (s.end_datetime <= %s OR s.start_datetime >= %s)
    """, (employee_id, new_start, new_end))
    (cnt,) = cursor.fetchone()
    cursor.close()
    conn.close()
    return cnt > 0

def employee_shifts_in_week(employee_id, ref_date):
    """
    ref_date: a datetime.date object representing the shift's date.
    returns count of shifts assigned to employee in that week (Mon-Sun)
    """
    week_start = ref_date - timedelta(days=ref_date.weekday())
    week_end = week_start + timedelta(days=6)
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT COUNT(*) FROM assignments a
        JOIN shifts s ON a.shift_id = s.id
        WHERE a.employee_id=%s
          AND DATE(s.start_datetime) BETWEEN %s AND %s
    """, (employee_id, week_start, week_end))
    (cnt,) = cursor.fetchone()
    cursor.close()
    conn.close()
    return cnt

def assign_all_shifts():
    # get all shifts ordered by start time
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT id, start_datetime, end_datetime, role_required FROM shifts ORDER BY start_datetime")
    shifts = cursor.fetchall()
    cursor.close()
    conn.close()

    # prepare round-robin cycles per role
    crew_ids = get_employees_by_role("Crew")
    mgr_ids  = get_employees_by_role("Manager")
    crew_cycle = itertools.cycle(crew_ids) if crew_ids else iter([])
    mgr_cycle  = itertools.cycle(mgr_ids) if mgr_ids else iter([])

    # To make assignment fairer, we will attempt up to N candidates per shift
    for sh in shifts:
        shift_id, s_start, s_end, role = sh
        assigned = False
        candidates = crew_ids if role.lower()=="crew" else mgr_ids
        # try each candidate up to len(candidates) times (round-robin fairness)
        for _ in range(len(candidates)):
            # pick next candidate from the appropriate cycle
            candidate = next(crew_cycle) if role.lower()=="crew" else next(mgr_cycle)
            # check overlap
            if employee_has_overlap(candidate, s_start, s_end):
                continue
            # check max 5 shifts in that week
            cnt = employee_shifts_in_week(candidate, s_start.date())
            if cnt >= 5:
                continue
            # assign
            try:
                conn = get_connection()
                cursor = conn.cursor()
                cursor.execute("INSERT INTO assignments (shift_id, employee_id) VALUES (%s, %s)", (shift_id, candidate))
                conn.commit()
                cursor.close()
                conn.close()
                assigned = True
                break
            except Error as e:
                # possible race or duplicate; skip candidate
                if conn and conn.is_connected():
                    cursor.close()
                    conn.close()
                continue
        if not assigned:
            # try all crew (fallback) just to ensure coverage: try any crew if role was crew, or any manager if manager
            # If still not assigned, leave unassigned (so you can see gaps in Power BI)
            pass

    print("Assignment pass completed.")

# Run the assignment
assign_all_shifts()

✅ Assignment pass completed.


In [7]:
# export full roster and weekly rosters
def export_roster_date_range(start_date, end_date, out_csv="roster_all_weeks.csv"):
    conn = None
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("""
            SELECT s.id, s.start_datetime, s.end_datetime, s.role_required,
                   e.id AS employee_id, e.name AS employee_name, e.role AS employee_role
            FROM shifts s
            LEFT JOIN assignments a ON s.id = a.shift_id
            LEFT JOIN employees e ON a.employee_id = e.id
            WHERE DATE(s.start_datetime) BETWEEN %s AND %s
            ORDER BY s.start_datetime
        """, (start_date, end_date))
        rows = cursor.fetchall()

        with open(out_csv, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["shift_id","start","end","role_required","employee_id","employee_name","employee_role"])
            for r in rows:
                writer.writerow(r)
        print("Exported:", out_csv)
    except Error as e:
        print("Error exporting roster:", e)
    finally:
        if conn and conn.is_connected():
            cursor.close()
            conn.close()

# Export full 3-week range
export_roster_date_range(start_date, end_date, "roster_all_weeks.csv")

# Also export per-week files (Monday start)
current = start_date
while current <= end_date:
    week_start = current
    week_end = week_start + timedelta(days=6)
    filename = f"roster_{week_start.isoformat()}_to_{week_end.isoformat()}.csv"
    export_roster_date_range(week_start, week_end, filename)
    current = current + timedelta(days=7)

print("All exports done. Files are in your notebook folder.")

✅ Exported: roster_all_weeks.csv
✅ Exported: roster_2025-09-21_to_2025-09-27.csv
✅ Exported: roster_2025-09-28_to_2025-10-04.csv
✅ Exported: roster_2025-10-05_to_2025-10-11.csv
✅ All exports done. Files are in your notebook folder.
